In [283]:
import pandas as pd
from elo import Elo

In [284]:
elo = Elo()

In [285]:
iturrizaga = 2665
mamedyarov = 2797

In [286]:
mo = elo.material_odds(iturrizaga, mamedyarov) # Iturrizaga v Mamedyarov

In [287]:
print(f'material odds {mo:.2f} pawns')

material odds -0.27 pawns


In [288]:
elo.probabilities(iturrizaga, mamedyarov)

{'black': 0.43506267422701833,
 'draw': 0.43740072992643048,
 'white': 0.12753659584655122}

In [289]:
delta = iturrizaga - mamedyarov

In [290]:
1 / (1 + 10**(-delta / 400))

0.3186781648336063

In [291]:
df = pd.concat([pd.read_csv('Super Saturday+Sunday Pairings - Super Saturday.csv').assign(Day='Super Saturday'),
                pd.read_csv('Super Saturday+Sunday Pairings - Super Sunday.csv').assign(Day='Super Sunday')])

In [292]:
df.head()

,Round,Round offset,Column offset,Row offset,Board offset,Team 1,Team 1 full name,Team 1 division,Player 1,Rating 1,Team 2,Team 2 full name,Team 2 division,Player 2,Rating 2,Day
0,1,6,0,0,0,Miami,Miami Champions,Atlantic,Eduardo Iturrizaga,2665,San Jose,San Jose Hackers,Pacific,Shakhriyar Mamedyarov,2797,Super Saturday
1,1,6,0,0,1,Miami,Miami Champions,Atlantic,Yuniesky Quesada,2619,San Jose,San Jose Hackers,Pacific,Rauf Mamedov,2678,Super Saturday
2,1,6,0,0,2,Miami,Miami Champions,Atlantic,Michael Rohde,2387,San Jose,San Jose Hackers,Pacific,Zviad Izoria,2601,Super Saturday
3,1,6,0,0,3,Miami,Miami Champions,Atlantic,Nicholas Rosenthal,2164,San Jose,San Jose Hackers,Pacific,Vinesh Ravuri,1918,Super Saturday
4,1,6,1,0,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Chengdu,Chengdu Pandas,Pacific,Yu Yangyi,2750,Super Saturday


In [293]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 512 entries, 0 to 255
Data columns (total 16 columns):
Round               512 non-null int64
Round offset        512 non-null int64
Column offset       512 non-null int64
Row offset          512 non-null int64
Board offset        512 non-null int64
Team 1              512 non-null object
Team 1 full name    512 non-null object
Team 1 division     512 non-null object
Player 1            512 non-null object
Rating 1            512 non-null int64
Team 2              512 non-null object
Team 2 full name    512 non-null object
Team 2 division     512 non-null object
Player 2            512 non-null object
Rating 2            512 non-null int64
Day                 512 non-null object
dtypes: int64(7), object(9)
memory usage: 68.0+ KB


In [294]:
df[df['Player 1'] == 'Fabiano Caruana']

,Round,Round offset,Column offset,Row offset,Board offset,Team 1,Team 1 full name,Team 1 division,Player 1,Rating 1,Team 2,Team 2 full name,Team 2 division,Player 2,Rating 2,Day
4,1,6,1,0,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Chengdu,Chengdu Pandas,Pacific,Yu Yangyi,2750,Super Saturday
68,3,12,1,6,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Seattle,Seattle Sluggers,Pacific,Hikaru Nakamura,2781,Super Saturday
132,5,18,1,12,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,San Diego,San Diego Surfers,Pacific,Alexey Dreev,2648,Super Saturday
196,7,24,1,18,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Rio Grande,Rio Grande Ospreys,Pacific,Hovhannes Gabuzyan,2573,Super Saturday


In [295]:
def score(x):
    # Player 1 is white on board 1 and 3 (board offset 0 and 2)
    p = x['Board offset'] % 2 == 0
    white_elo = x['Rating 1'] if p else x['Rating 2']
    black_elo = x['Rating 2'] if p else x['Rating 1']
    pr = elo.probabilities(white_elo, black_elo)
    return pd.Series({'Player 1 score': (pr['white'] if p else pr['black']) + pr['draw'] / 2,
                      'Player 2 score': (pr['black'] if p else pr['white']) + pr['draw'] / 2})

score_df = pd.concat([df, df.apply(score, axis=1)], axis=1)

In [296]:
pd.concat([score_df[score_df['Player 1'] == 'Fabiano Caruana'],
           score_df[score_df['Player 2'] == 'Fabiano Caruana']])

,Round,Round offset,Column offset,Row offset,Board offset,Team 1,Team 1 full name,Team 1 division,Player 1,Rating 1,Team 2,Team 2 full name,Team 2 division,Player 2,Rating 2,Day,Player 1 score,Player 2 score
4,1,6,1,0,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Chengdu,Chengdu Pandas,Pacific,Yu Yangyi,2750,Super Saturday,0.600832,0.399168
68,3,12,1,6,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Seattle,Seattle Sluggers,Pacific,Hikaru Nakamura,2781,Super Saturday,0.561195,0.438805
132,5,18,1,12,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,San Diego,San Diego Surfers,Pacific,Alexey Dreev,2648,Super Saturday,0.722907,0.277093
196,7,24,1,18,0,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Rio Grande,Rio Grande Ospreys,Pacific,Hovhannes Gabuzyan,2573,Super Saturday,0.800526,0.199474
36,2,9,1,3,0,Dallas,Dallas Destiny,Pacific,Jeffery Xiong,2633,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Super Saturday,0.302229,0.697771
100,4,15,1,9,0,San Jose,San Jose Hackers,Pacific,Shakhriyar Mamedyarov,2797,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Super Saturday,0.536254,0.463746
164,6,21,1,15,0,Las Vegas,Las Vegas Desert Rats,Pacific,Leinier Dominguez,2739,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Super Saturday,0.449866,0.550134
228,8,27,1,21,0,Australia,Australia Kangaroos,Pacific,Evgenij Miroshnichenko,2606,St. Louis,St. Louis Arch Bishops,Atlantic,Fabiano Caruana,2799,Super Saturday,0.267794,0.732206


In [297]:
# Sum score by player (+ day, division, board, team)
total_df = pd.concat([
    score_df.apply(lambda x: pd.Series({'Day': x['Day'], 'Division': x['Team 1 division'], 'Board offset': x['Board offset'], 
                                        'Team': x['Team 1 full name'], 'Player': x['Player 1'], 'Score': x['Player 1 score']}), axis=1),
    score_df.apply(lambda x: pd.Series({'Day': x['Day'], 'Division': x['Team 2 division'], 'Board offset': x['Board offset'],
                                        'Team': x['Team 2 full name'], 'Player': x['Player 2'], 'Score': x['Player 2 score']}), axis=1)]) \
    .groupby(['Day', 'Division', 'Board offset', 'Team', 'Player']).sum()

In [298]:
total_df.head()

Score
Day            Division Board offset Team                 Player                       
Super Saturday Atlantic 0            Buenos Aires Krakens Alan Pichot          2.692616
                                     Miami Champions      Eduardo Iturrizaga   3.724052
                                     Minnesota Blizzard   Andrew Tang          1.984620
                                     Montclair Sopranos   Aleksandr Lenderman  2.692616
                                     Montreal Chessbrahs  Eric Hansen          3.346430

In [299]:
# Best player per division and board
total_df.groupby(level=[0, 1, 2], group_keys=False).apply(lambda x: x.sort_values('Score', ascending=False).head(1))

Score
Day            Division Board offset Team                   Player                          
Super Saturday Atlantic 0            St. Louis Arch Bishops Fabiano Caruana         5.129317
                        1            St. Louis Arch Bishops Vladimir Fedoseev       5.519289
                        2            St. Louis Arch Bishops Alejandro Ramirez       4.667070
                        3            Montclair Sopranos     Nicolas Checa           6.079835
               Pacific  0            San Jose Hackers       Shakhriyar Mamedyarov   5.847165
                        1            San Jose Hackers       Rauf Mamedov            5.570475
                        2            San Jose Hackers       Zviad Izoria            5.682621
                        3            Rio Grande Ospreys     Felix Ynojosa           4.897751
Super Sunday   Central  0            Marseille Migraines    Maxime Vachier-Lagrave  5.989152
                        1            Stockholm Snowballs    Levan Pantsulaia        5.047084
                        2            Stockholm Snowballs    Kirill Alekseenko       5.670920
                        3            Reykjavik Puffins      Bjorn Thorfinnsson      5.555969
               Eastern  0            Norway Gnomes          Magnus Carlsen          6.208968
                        1            Armenia Eagles         Zaven Andriasyan        4.885456
                        2            Armenia Eagles         Manuel Petrosyan        4.964257
                        3            Mumbai Movers          Raunak Sadhwani         4.899887

In [300]:
# Best team per day
total_df.groupby(level=['Day', 'Team'], group_keys=False).sum() \
    .groupby(level='Day', group_keys=False).apply(lambda x: x.sort_values('Score', ascending=False).head(1))

,,Score
Day,Team,
Super Saturday,Chengdu Pandas,18.575540
Super Sunday,Stockholm Snowballs,19.633989


In [301]:
# Rating by player (+ day, division, board, team)
rating_df = pd.concat([
    score_df.apply(lambda x: pd.Series([x['Day'], x['Team 1 division'], x['Board offset'], x['Team 1 full name'], x['Player 1'], x['Rating 1']],
                                       index=['Day', 'Division', 'Board offset', 'Team', 'Player', 'Rating']), axis=1),
    score_df.apply(lambda x: pd.Series([x['Day'], x['Team 2 division'], x['Board offset'], x['Team 2 full name'], x['Player 2'], x['Rating 2']],
                                       index=['Day', 'Division', 'Board offset', 'Team', 'Player', 'Rating']), axis=1)]) \
    .drop_duplicates()

In [302]:
rating_df[rating_df['Player'] == 'Fabiano Caruana']

,Day,Division,Board offset,Team,Player,Rating
4,Super Saturday,Atlantic,0,St. Louis Arch Bishops,Fabiano Caruana,2799


In [303]:
(rating_df.groupby('Player').count() == 1).all()

Day             True
Division        True
Board offset    True
Team            True
Rating          True
dtype: bool

In [304]:
# Best rated player by division and board
rating_df.groupby(['Day', 'Division', 'Board offset']).apply(lambda x: x.sort_values('Rating', ascending=False).head(1))

Day  Division  \
Day            Division Board offset                                
Super Saturday Atlantic 0            4   Super Saturday  Atlantic   
                        1            5   Super Saturday  Atlantic   
                        2            6   Super Saturday  Atlantic   
                        3            51  Super Saturday  Atlantic   
               Pacific  0            44  Super Saturday   Pacific   
                        1            45  Super Saturday   Pacific   
                        2            46  Super Saturday   Pacific   
                        3            31  Super Saturday   Pacific   
Super Sunday   Central  0            20    Super Sunday   Central   
                        1            41    Super Sunday   Central   
                        2            42    Super Sunday   Central   
                        3            27    Super Sunday   Central   
               Eastern  0            8     Super Sunday   Eastern   
                        1            61    Super Sunday   Eastern   
                        2            62    Super Sunday   Eastern   
                        3            55    Super Sunday   Eastern   

                                         Board offset                    Team  \
Day            Division Board offset                                            
Super Saturday Atlantic 0            4              0  St. Louis Arch Bishops   
                        1            5              1  St. Louis Arch Bishops   
                        2            6              2  St. Louis Arch Bishops   
                        3            51             3      Montclair Sopranos   
               Pacific  0            44             0        San Jose Hackers   
                        1            45             1        San Jose Hackers   
                        2            46             2        San Jose Hackers   
                        3            31             3      Rio Grande Ospreys   
Super Sunday   Central  0            20             0     Marseille Migraines   
                        1            41             1     Stockholm Snowballs   
                        2            42             2     Stockholm Snowballs   
                        3            27             3       Reykjavik Puffins   
               Eastern  0            8              0           Norway Gnomes   
                        1            61             1          Armenia Eagles   
                        2            62             2          Armenia Eagles   
                        3            55             3           Mumbai Movers   

                                                         Player  Rating  
Day            Division Board offset                                     
Super Saturday Atlantic 0            4          Fabiano Caruana    2799  
                        1            5        Vladimir Fedoseev    2731  
                        2            6        Alejandro Ramirez    2563  
                        3            51           Nicolas Checa    2459  
               Pacific  0            44   Shakhriyar Mamedyarov    2797  
                        1            45            Rauf Mamedov    2678  
                        2            46            Zviad Izoria    2601  
                        3            31           Felix Ynojosa    2418  
Super Sunday   Central  0            20  Maxime Vachier-Lagrave    2804  
                        1            41        Levan Pantsulaia    2593  
                        2            42       Kirill Alekseenko    2563  
                        3            27      Bjorn Thorfinnsson    2398  
               Eastern  0            8           Magnus Carlsen    2827  
                        1            61        Zaven Andriasyan    2593  
                        2            62        Manuel Petrosyan    2559  
                        3            55         Raunak Sadhwani    2339

In [305]:
# Some rating statistics per team
rating_df.groupby(['Day', 'Division', 'Team']).describe()['Rating'].sort_values('mean', ascending=False)

count     mean         std  \
Day            Division Team                                                  
Super Saturday Atlantic St. Louis Arch Bishops     4.0  2545.75  319.613699   
               Pacific  Chengdu Pandas             4.0  2536.50  256.110002   
Super Sunday   Central  Stockholm Snowballs        4.0  2519.50  169.645709   
Super Saturday Pacific  Seattle Sluggers           4.0  2511.50  259.216126   
Super Sunday   Eastern  Volga Stormbringers        4.0  2501.00  172.373625   
Super Saturday Pacific  Rio Grande Ospreys         4.0  2499.50   81.422765   
                        San Jose Hackers           4.0  2498.50  395.309583   
Super Sunday   Eastern  Delhi Dynamite             4.0  2497.75  145.756361   
                        Armenia Eagles             4.0  2497.50  203.866787   
Super Saturday Atlantic Montclair Sopranos         4.0  2496.25   48.616698   
               Pacific  Australia Kangaroos        4.0  2493.50  123.251775   
                        Dallas Destiny             4.0  2489.50  148.055170   
               Atlantic Webster Windmills          4.0  2488.25  118.643373   
               Pacific  Las Vegas Desert Rats      4.0  2487.75  251.480118   
                        San Diego Surfers          4.0  2487.00  119.082604   
Super Sunday   Central  Amsterdam Mosquitoes       4.0  2478.50  125.162561   
Super Saturday Atlantic Montreal Chessbrahs        4.0  2478.50  107.546889   
Super Sunday   Central  Marseille Migraines        4.0  2470.75  366.771841   
                        Reykjavik Puffins          4.0  2469.50   53.200877   
Super Saturday Atlantic Minnesota Blizzard         4.0  2463.00   20.379728   
Super Sunday   Central  Ljubljana Turtles          4.0  2461.50  236.701641   
               Eastern  Mumbai Movers              4.0  2459.25  102.532515   
Super Saturday Atlantic Miami Champions            4.0  2458.75  231.116673   
Super Sunday   Central  London Lions               4.0  2456.00   70.323064   
Super Saturday Atlantic Pittsburgh Pawngrabbers    4.0  2449.00   81.698633   
                        Buenos Aires Krakens       4.0  2429.00  100.773674   
Super Sunday   Eastern  Norway Gnomes              4.0  2427.75  393.955475   
               Central  Cannes Blockbusters        4.0  2405.75  132.303628   
               Eastern  Riga Magicians             4.0  2386.25  113.129351   
                        Oslo Trolls                4.0  2386.00   40.743098   
               Central  London Towers              4.0  2365.00   78.710440   
               Eastern  Estonia Horses             4.0  2254.50  151.669597   

                                                    min      25%     50%  \
Day            Division Team                                               
Super Saturday Atlantic St. Louis Arch Bishops   2090.0  2444.75  2647.0   
               Pacific  Chengdu Pandas           2173.0  2455.00  2611.5   
Super Sunday   Central  Stockholm Snowballs      2271.0  2490.00  2578.0   
Super Saturday Pacific  Seattle Sluggers         2169.0  2403.00  2548.0   
Super Sunday   Eastern  Volga Stormbringers      2305.0  2401.00  2495.5   
Super Saturday Pacific  Rio Grande Ospreys       2418.0  2435.25  2503.5   
                        San Jose Hackers         1918.0  2430.25  2639.5   
Super Sunday   Eastern  Delhi Dynamite           2287.0  2465.50  2542.5   
                        Armenia Eagles           2196.0  2468.25  2576.0   
Super Saturday Atlantic Montclair Sopranos       2459.0  2463.50  2480.5   
               Pacific  Australia Kangaroos      2319.0  2460.75  2524.5   
                        Dallas Destiny           2292.0  2424.00  2516.5   
               Atlantic Webster Windmills        2358.0  2445.00  2474.5   
               Pacific  Las Vegas Desert Rats    2139.0  2429.25  2536.5   
                        San Diego Surfers        2373.0  2414.25  2463.5   
Super Sunday   Central  Amsterdam Mosquitoes     2300.0  2443.25  2514

In [306]:
# Best team by rating (Pandas expected to score more than Arch Bishops... better lower boards?)
rating_df.groupby(['Day', 'Division', 'Team']).mean() \
    .groupby(level='Day', group_keys=False).apply(lambda x: x.sort_values('Rating', ascending=False).head(1))

,,,Board offset,Rating
Day,Division,Team,,
Super Saturday,Atlantic,St. Louis Arch Bishops,1.5,2545.75
Super Sunday,Central,Stockholm Snowballs,1.5,2519.50
